# HW 2

#### Imports

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler,StandardScaler,QuantileTransformer
from scipy.spatial.distance import cdist
from sklearn import linear_model, datasets
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge

In [2]:
import Lowess
import Logistic as LowessLR
import g_boost

In [3]:
#!pip install xgboost

#### Kernels

In [4]:
# Defining Kernels

# Gaussian Kernel
def Gaussian(x):
  return np.where(np.abs(x)>4,0,1/(np.sqrt(2*np.pi))*np.exp(-1/2*x**2))
    
# Tricubic Kernel
def Tricubic(x):
  return np.where(np.abs(x)>1,0,(1-np.abs(x)**3)**3)
    
# Epanechnikov Kernel
def Epanechnikov(x):
  return np.where(np.abs(x)>1,0,3/4*(1-np.abs(x)**2))
    
# Quartic Kernel
def Quartic(x):
  return np.where(np.abs(x)>1,0,15/16*(1-np.abs(x)**2)**2)

## Part 1

Create your class that implements the Gradient Boosting concept, based on the locally weighted regression method (Lowess class), and that allows a user-prescribed number of boosting steps. The class you develop should have all the mainstream useful options, including “fit,” “is_fitted”, and “predict,” methods. Show applications with real data for regression, 10-fold cross-validations and compare the effect of different scalers, such as the “StandardScaler”, “MinMaxScaler”, and the “QuantileScaler”. In the case of the “Concrete” data set, determine a choice of hyperparameters that yield lower MSEs for your method when compared to the eXtream Gradient Boosting library.

In [5]:
#data = pd.read_csv('../data/concrete.csv')
data = pd.read_csv('./data/concrete.csv')

X = data.drop(columns='strength').values
y = data['strength'].values

In [6]:
# Scalers to evaluate
scalers = {
    "StandardScaler": StandardScaler(),
    "MinMaxScaler": MinMaxScaler(),
    "QuantileScaler": QuantileTransformer(n_quantiles=min(900, X.shape[0]), output_distribution='normal')
}

In [7]:
results = {}

for scaler_name, scaler in scalers.items():
    X_scaled = scaler.fit_transform(X)

    # Initialize 10-fold cross-validation
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    
    mse_scores = []

    # Perform cross-validation
    for train_index, test_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Instantiate models
        lowess_model = Lowess.Lowess(kernel=Gaussian)
        ridge_model = Ridge(alpha=0.001)
        booster = g_boost.GradientBooster(lowess_model, ridge_model, boosting_steps=10)
        
        # Fit the model
        booster.fit(X_train, y_train)
        
        # Predict and evaluate
        y_pred = booster.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        mse_scores.append(mse)

    # Store the mean MSE for the current scaler
    results[scaler_name] = np.mean(mse_scores)

In [8]:
for scaler_name, mse_value in results.items():
    print(f"{scaler_name}: MSE = {mse_value:.4f}")

StandardScaler: MSE = 860.2757
MinMaxScaler: MSE = 101.7934
QuantileScaler: MSE = 951.1050


#### eXtreme Gradient Boosting

In [9]:
# initializing list to store results of XGBoost
xgboost_results = []

# looping through each scaler in scaler dictionary from first part
for scaler_name, scaler in scalers.items():
    # initializing list to store MSE scores for each fold in cross-validation
    mse_scores = []

    # performing k-fold cross-validation
    for train_index, test_index in kf.split(X):
        # splitting the dataset into training and testing sets based on indices
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # scaling training data based on current scaler
        X_train_scaled = scaler.fit_transform(X_train)
        # scaling testing data using the same scaler
        X_test_scaled = scaler.transform(X_test)

        
        # creating DMatrix for XGBoost (data structure optimized for XGBoost)
        dtrain = xgb.DMatrix(X_train_scaled, label=y_train)
        dtest = xgb.DMatrix(X_test_scaled, label=y_test)

        # setting parameters for XGBoost model
        params = {
            'objective': 'reg:squarederror', # objective function for regression
            'max_depth': 3,    # maximum tree depth, maximum number of levels a tree can have
            'learning_rate': 0.001,  # learning rate
        }

        # training the XGBoost model with the specified parameters
        xgboost_model = xgb.train(params, dtrain, num_boost_round=1000)

        # making predictions on test set
        y_pred_xgb = xgboost_model.predict(dtest)

        # calculating mse for predictions
        mse_xgb = mean_squared_error(y_test, y_pred_xgb)
        # adding MSE score to the list of scores
        mse_scores.append(mse_xgb)

    # adding the mean MSE score for this scaler to the results list
    xgboost_results.append(np.mean(mse_scores))

# displaying the average MSE results for each scaler
for scaler_name, mse in zip(scalers.keys(), xgboost_results):
    print(f"XGBoost with {scaler_name}: Mean MSE = {mse:.4f}")

XGBoost with StandardScaler: Mean MSE = 111.2413
XGBoost with MinMaxScaler: Mean MSE = 111.2413
XGBoost with QuantileScaler: Mean MSE = 111.2413


## Part 2
Implement your own version of Locally Weighted Logistic Regression and compare its performance on the Iris data set with the version presented in this article: https://calvintchi.github.io/classical_machine_learning/2020/08/16/lwlr.html.

### Importing iris data, TTS, and Scaling

In [10]:
# importing data from sklearn
iris = datasets.load_iris()

In [11]:
# defining x and y variables
X = iris.data
y = iris.target

In [12]:
# performing test-train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [13]:
# scaling data using MinMax Scaler, can also use other scalers which are currently commented out
scaler = MinMaxScaler()
#scaler = StandardScaler()
#scaler = QuantileTransformer()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Model Training and Evaluation

In [14]:
# creating a LocallyWeightedLogisticRegression model with Gaussian kernel, learing rate of  0.01, and tau=0.05
model = LowessLR.LocallyWeightedLogisticRegression(kernel=Gaussian, lr=0.01, tau=0.05) # accuracy of .97

#model = LowessLR.LocallyWeightedLogisticRegression(kernel=Tricubic, lr=0.01, tau=0.05) # accuracy of .97
#model = LowessLR.LocallyWeightedLogisticRegression(kernel=Epanechnikov, lr=0.01, tau=0.05) # accuracy of .97
#model = LowessLR.LocallyWeightedLogisticRegression(kernel=Quartic, lr=0.01, tau=0.05) # accuracy of .97

In [15]:
# fitting model to training data
model.fit(X_train, y_train)

In [16]:
# using trained model to make predictions on test data
predictions = model.predict(X_test)

In [17]:
# Calculating accuracy of predictions compared to the true labels (y_test) and printing results
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 96.67%


### Comparing results to Calvin Chi Paper

In [18]:
# version presented in Calvin Chi article

class locally_weighted_logistic_regression(object):
    
    def __init__(self, tau, reg = 0.0001, threshold = 1e-6):
        self.reg = reg
        self.threshold = threshold
        self.tau = tau
        self.w = None
        self.theta = None
        self.x = None

    def weights(self, x_train, x):
        sq_diff = (x_train - x)**2
        norm_sq = sq_diff.sum(axis = 1)
        return np.ravel(np.exp(- norm_sq / (2 * self.tau**2)))

    def logistic(self, x_train):
        return np.ravel(1 / (1 + np.exp(-x_train.dot(self.theta))))

    def train(self, x_train, y_train, x):
        self.w = self.weights(x_train, x)
        self.theta = np.zeros(x_train.shape[1])
        self.x = x
        gradient = np.ones(x_train.shape[1]) * np.inf
        while np.linalg.norm(gradient) > self.threshold:
            # compute gradient
            h = self.logistic(x_train)
            gradient = x_train.T.dot(self.w * (np.ravel(y_train) - h)) - self.reg * self.theta
            # Compute Hessian
            D = np.diag(-(self.w * h * (1 - h)))
            H = x_train.T.dot(D).dot(x_train) - self.reg * np.identity(x_train.shape[1])
            # weight update
            self.theta = self.theta - np.linalg.inv(H).dot(gradient)
    
    def predict(self,x):  # adjusted slightly to allow for input feature
        return np.array(self.logistic(x) > 0.5).astype(int)

In [19]:
# training one v rest models
model_dict = {}  # initialize dictionary to store models

# training a model for each class
for cls in np.unique(y_train):  # iterating over each unique class label in the training data
    binary_y_train = (y_train == cls).astype(int)  # creating binary target variable for current class
    model = locally_weighted_logistic_regression(tau=.05)  # initializing model
    model.train(X_train, binary_y_train, X_train)  # training  model
    model_dict[cls] = model  # storing model for given class

In [20]:
# making predictions
predictions = []  # initializing list to store predictions

for x_test in X_test: # iterating over each test sample
    class_probs = [] # initializing list to store probabilities for each class
    
    for cls, model in model_dict.items():  # iterating over the items in model_dict
        prob = model.predict(x_test.reshape(1, -1))  # calling predict method of current model to get prediction
        class_probs.append(prob[0])  # appends predicted probability for current class to class_probs list
    
    # after evaluating all classes for current test sample, determine class with highest probability
    predictions.append(np.argmax(class_probs))

In [21]:
# Calculating accuracy of predictions compared to the true labels (y_test) and printing results

accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 83.33%
